In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import itertools
import string
from nltk.corpus import stopwords


In [2]:
sg = pickle.load(open('sg_post_df.pkl', 'br'))
sg = sg[['post_number', 'post_txt', 'groups']]
sg = sg.drop_duplicates()
sg = sg.reset_index(drop=True)


In [3]:
sg.sample(5)


,post_number,post_txt,groups
20589,343200,"['n How do you deal with family, siblings, wh...","[' Relationship, Family, Siblings']"
25018,119277,"['n ', 'I was recely diagnosed with PD at 41 ...",[' Parkinsons']
21844,159267,"['n ', 'Good morning everyone,,it is 8u00272...",[' Fibromyalgia']
29552,148574,"['n ', 'WE MADE IT HEZEKIAH WALKER: ', 'n', '...","[' Bipolar, Depression, Social Anxiety']"
26930,135280,['n My dog just died today. he wasnu0027t ev...,[' Pet Loss']


In [4]:
sg['post_number'] = sg['post_number'].astype(int)
sg['groups'] = sg['groups'].str.strip('[]\'').str.split(',')
sg['post_txt'] = sg['post_txt'].str.strip('[]\'')
sg.sample(5)

,post_number,post_txt,groups
16145,210688,"n Just got back, .....thought I could do sams...","[ Stroke, Weight Loss]"
400,601891,n Iu2019ve had acne for almost 3 years now an...,[ Acne]
7302,583275,"n ', 'My doctor recely put me on 90 mgs of Cy...",[ Cymbalta]
3232,530947,n Hello. Iu0027m new to the group. I saw ther...,[ Arthritis]
28257,49435,"n ', 'Iu0027m in pain and i need to know if i...",[ Hyperthyroidism]


In [5]:
#pickle.dump(sg, open('sg_post_dfv2.pkl', 'bw'))

In [6]:
sg['post_txt'][1]

"n  I feel so trapped and helpless! My husband has borderline personality disorder and when he is well he is so lovely and caring but when he is feeling ill he emotionally abuses me, swears at me, blames me for everything, tells me Iu0027m useless and I am g...', '', "

In [7]:
sg.post_txt = sg.post_txt.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
sg.post_txt[1]

'n  I feel so trapped and helpless My husband has borderline personality disorder and when he is well he is so lovely and caring but when he is feeling ill he emotionally abuses me swears at me blames me for everything tells me Iu0027m useless and I am g  '

In [8]:
sg = sg.reset_index(drop=True)
sg.sample(10)


,post_number,post_txt,groups
2628,612734,n Having fun with the little one even though ...,"[ Anger Management, Single Moms, Single Pare..."
34042,50277,n Please help me I canu0027t type I canu002...,"[ Self Harm, Suicide, Self Esteem]"
28634,110856,n Hi This is my 1st time on here and post I a...,"[ Loneliness, Relationship, Single Dads]"
26712,157279,n Itu0027s a new day My tactic not to jealous...,[ Jealousy]
17174,359747,n Hello I new to the group and Iu0027m looki...,[ Healthy Eating]
15771,413112,n Hey so I just received a call from my docto...,[ HPV]
5571,612946,n Hi everyonenI was diagnosed with rosacea th...,[ Rosacea]
29173,155995,n I have recely transitioned from a vegetaria...,[ Vegan]
29130,148544,n I have no idea how to or if i should come o...,[ Coming Out]
3211,94896,n Just curious if there is anything anyone t...,[ Arthritis]


In [9]:
grp_sp = pd.DataFrame(sg.groups.values.tolist())
grp_sp.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
4954,Dizziness,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
22373,Depression,Infertility,PCOS,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9456,Video Game Addiction,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
27916,Trichotillomania,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
32612,Abuse,Anorexia,Anxiety,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
23524,Miscarriage,Obesity,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
21284,Binge Eating,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1256,Financial Problems,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2046,Codependency,Relationship,Self Esteem,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
21390,Borderline Personality Disorder,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
sg['group_1'] = grp_sp[0]
sg['group_2'] = grp_sp[1]
sg['group_3'] = grp_sp[2]
sg['group_4'] = grp_sp[3]
sg['group_5'] = grp_sp[4]
sg['group_1'] = sg['group_1'].str.strip()
sg['group_1'] = sg['group_1'].str.replace("'", "")
sg.sample(10)


,post_number,post_txt,groups,group_1,group_2,group_3,group_4,group_5
17883,330841,n,[ Cymbalta],Cymbalta,None,None,None,None
5331,576887,n My wife passed away on August 15 2018 We w...,[ Widower],Widower,None,None,None,None
3247,614797,n Hi everyone one I am an hub patie been di...,[ HIV],HIV,None,None,None,None
25204,67413,n Any one recely suffering from have this ras...,[ Eczema],Eczema,None,None,None,None
12606,399745,n DOES ANYONE HAVE PROBLEMS WITH SEVERE BURN...,[ Fibromyalgia],Fibromyalgia,None,None,None,None
24454,143356,n just joined checking to see it reg we thro...,[ Migraine],Migraine,None,None,None,None
3960,622118,n,[ Asperger Syndrome],Asperger Syndrome,None,None,None,None
33839,47525,n My name is Joe and I am 33 I have smoked a...,[ Chantix],Chantix,None,None,None,None
387,600430,n Iu0027m 17 This is my 4th day using Celeteq...,[ Acne],Acne,None,None,None,None
5160,618944,n My family is extremely religious I really l...,[ Teen],Teen,None,None,None,None


In [14]:
pickle.dump(sg, open('sg_post_dfv3.pkl', 'bw'))

In [19]:
group_names = pd.unique(sg['group_1'])
group_names

array(['Friends and Family of BPD', 'Divorce',
       'Borderline Personality Disorder', 'Depression', 'Caregivers',
       'Transgender', 'Self Harm', 'Alcohol', 'ADHD', 'Emotional Abuse',
       'Teen', 'Gay and Lesbian', 'Relationship', 'Coming Out',
       'Retirement', 'Anxiety', 'Irritable Bowel Syndrome',
       'Friends and Family of Sex Addicts', 'Drug', 'Bulimia', 'Cocaine',
       'Back Pain', 'Bipolar', 'Marijuana', 'Abuse', 'Sexual Abuse',
       'Acne', 'Body Dysmorphic Disorder', 'Adderall', 'Moms', 'Anorexia',
       'Binge Eating', 'Ambien', 'Asperger Syndrome', 'Autism',
       'Loneliness', 'Anger Management', 'Family', 'Breast Cancer',
       'Brain Cancer', 'College', 'Bullying', 'Grief', 'Diabetes',
       'Birth Defects', 'Shopping Addiction', 'Obesity',
       'Eating Disorder', 'Parents', 'Dads', 'Narcissist',
       'Financial Problems', 'Career Changes', 'Hoarding',
       'Chronic Fatigue Syndrome', 'Food Allergy', 'Asthma', 'Vegan',
       'Crohns Disease',

In [40]:
seed1 = 'https://'
seed2 = '.supportgroups.com'
lst = group_names

def url_dict(seed1, seed2,lst):
    dashes = []
    for i in range(0,len(lst)):
        dashes.append(lst[i].replace(" ", "-"))
    urls = []
    for j in range(0, len(dashes)):
        urls.append("\'" + dashes[j] + "\'" + ' : '  + "\'" + seed1 + dashes[j] + seed2+ "\'")
    return urls

dict_list = url_dict(seed1, seed2, lst)
dict_list = ",".join(dict_list)
dict_list
        
    
    
    


"'Friends-and-Family-of-BPD' : 'https://Friends-and-Family-of-BPD.supportgroups.com','Divorce' : 'https://Divorce.supportgroups.com','Borderline-Personality-Disorder' : 'https://Borderline-Personality-Disorder.supportgroups.com','Depression' : 'https://Depression.supportgroups.com','Caregivers' : 'https://Caregivers.supportgroups.com','Transgender' : 'https://Transgender.supportgroups.com','Self-Harm' : 'https://Self-Harm.supportgroups.com','Alcohol' : 'https://Alcohol.supportgroups.com','ADHD' : 'https://ADHD.supportgroups.com','Emotional-Abuse' : 'https://Emotional-Abuse.supportgroups.com','Teen' : 'https://Teen.supportgroups.com','Gay-and-Lesbian' : 'https://Gay-and-Lesbian.supportgroups.com','Relationship' : 'https://Relationship.supportgroups.com','Coming-Out' : 'https://Coming-Out.supportgroups.com','Retirement' : 'https://Retirement.supportgroups.com','Anxiety' : 'https://Anxiety.supportgroups.com','Irritable-Bowel-Syndrome' : 'https://Irritable-Bowel-Syndrome.supportgroups.com'

In [41]:
pickle.dump(dict_list, open('group_dict.pkl', 'bw'))